In [45]:
import matplotlib.pyplot as plt
from comtrade import Comtrade
import plotly.graph_objects as go
from antialising import Filtro
from array import array
from transformada import *
import secuencias
import numpy as np
import cmath

def closest(lst, K):
      
     lst = np.asarray(lst)
     idx = (np.abs(lst - K)).argmin()
     return lst[idx]


rec = Comtrade()
rec.load("sample_files/ATPmonoA06.cfg", "sample_files/ATPmonoA06.dat")
print("Trigger time = {}s".format(rec.trigger_time))

#Posiciones de las señales de voltaje y corriente en el COMTRADE
senal_I_a=0

senal_I_b=1

senal_I_c=2

senal_V_a=3

senal_V_b=4

senal_V_c=5

#Plot de la corriente de la fase a
""" plt.figure()
plt.plot(rec.time, rec.analog[senal_I_a])
plt.legend([rec.analog_channel_ids[senal_I_a]])
plt.show() """

#Simulacion de ventaneo a 10 muestras por ciclo.
def Windowing(senal_a,senal_b,senal_c,rec,I_nominal,I_sc3_min,V_a,V_b,V_c,k0,z1l):
    data_a=rec.analog[senal_a]
    data_b=rec.analog[senal_b]
    data_c=rec.analog[senal_c]
    data_Va=rec.analog[V_a]
    data_Vb=rec.analog[V_b]
    data_Vc=rec.analog[V_c]
    t=rec.time
    fs = rec.cfg.sample_rates[0][0]       # sample rate 'analogo', Hz 
    fr=720#Frecuencia del relee
    r=(1/fr)//(1/fs)#Cantidad de muestras en 1 ciclo de una señal de 60Hz

    I_phase_r=np.empty((7,len(range(0,len(data_a),int(r)))))#[Ia;Ib;Ic;t;Va;Vb;Vc]

    for i in range(0,len(data_a),int(r)):#Creacion del array de muestras que toma el rele.
        I_phase_r[0][int(i/int(r))]=data_a[i]
        I_phase_r[1][int(i/int(r))]=data_b[i]
        I_phase_r[2][int(i/int(r))]=data_c[i]
        I_phase_r[3][int(i/int(r))]=t[i]
        I_phase_r[4][int(i/int(r))]=data_Va[i]
        I_phase_r[5][int(i/int(r))]=data_Vb[i]
        I_phase_r[6][int(i/int(r))]=data_Vc[i]
    s=1#numero de ciclos por ventana
    w=int(s*fr/60)#numero de muestras por ventana
  
    last_msg="Sin falla"#ultimo mensaje leido
    n=0#Cantidades sucesivas que se ha leido ese mensaje. Con 6 ventanas sucesivas (medio ciclo) reportando la misma falla, se activa el mecanismo de deteccion de fase fallada.
    k=0

    fault=False
    while(w+k<=len(I_phase_r[3])):#Por cada ventana se evalua la posibilidad de falla. De haber una, se determina que fases son las falladas.
        I_a=Espectro(I_phase_r[0][k:w+k],fr)
        I_b=Espectro(I_phase_r[1][k:w+k],fr)
        I_c=Espectro(I_phase_r[2][k:w+k],fr)
        k+=1
        Seq=secuencias.seq(I_a,I_b,I_c)#Referidos a la fase a
        
        I0=Seq.item((0,0))
        I1=Seq.item((1,0))
        I2=Seq.item((2,0))
       
        msg=Evaluar(I0,I1,I2,I_nominal,I_sc3_min)
        #print(msg)
        if msg=="Sin falla":
            n=0
        elif msg==last_msg:
            n+=1
        else:
            last_msg=msg
            n=1
            
        if n==6 or fault:
            fault=True
            v_a=Espectro(I_phase_r[4][k:w+k],fr)
            v_b=Espectro(I_phase_r[5][k:w+k],fr)
            v_c=Espectro(I_phase_r[6][k:w+k],fr)
            DefinePhase(last_msg,I_a,I_b,I_c,v_a,v_b,v_c,I2,I0,k0,z1l)

#TODO Reemplazar los printos la ejecucion del calculo de la impedancia y la distancia ya conociendo las fases falladas.
def DefinePhase(msg,Ia,Ib,Ic,Va,Vb,Vc,I2,I0,k0,Z1l):
    if msg=="Falla BG o AC":
        MHO_BG=abs(Vb/(Ib+k0*3*I0))
        MHO_AC=abs((Va-Vc)/(Ia-Ic))
        if MHO_BG>=MHO_AC:
            print("Falla AC")
            Vg=Vc-Va
            Ig=Ic-Ia
            Iseq=I2*cmath.rect(1,2*np.pi/3)#Referenciada a la fase sana (B)
            m=np.imag(Vg*1j*np.conjugate(Iseq))/np.imag(Z1l*Ig*1j*np.conjugate(Iseq))
            print("Distancia(pu): "+str(m))
            
        else:
            print("Falla BG")
            Vg=Vb
            Ig=Ib+k0*I0*3
            Iseq=I2*cmath.rect(1,2*np.pi/3)
            m=np.imag(Vg*np.conjugate(Iseq))/np.imag(Z1l*Ig*np.conjugate(Iseq))
            print("Distancia(pu): "+str(m))
    elif msg=="Falla AG o BC":
        MHO_AG=abs(Va/(Ia+k0*3*I0))
        MHO_BC=abs((Vb-Vc)/(Ib-Ic))
        if MHO_AG>=MHO_BC:
            print("Falla BC")
            Vg=Vb-Vc
            Ig=Ib-Ic
            Iseq=I2
            m=np.imag(Vg*1j*np.conjugate(Iseq))/np.imag(Z1l*Ig*1j*np.conjugate(Iseq))
            print("Distancia(pu): "+str(m))
        else:
            print("Falla AG")
            Vg=Va
            Ig=Ia+k0*I0*3
            Iseq=I2
            m=np.imag(Vg*np.conjugate(Iseq))/np.imag(Z1l*Ig*np.conjugate(Iseq))
            print("Distancia(pu): "+str(m))
    elif msg=="Falla CG o BA":
        MHO_CG=abs(Vc/(Ic+k0*3*I0))
        MHO_AB=abs((Va-Vb)/(Ia-Ib))
        if MHO_CG>=MHO_AB:
            print("Falla AB")
            Vg=Va-Vb
            Ig=Ia-Ib
            Iseq=I2*cmath.rect(1,-2*np.pi/3)
            m=np.imag(Vg*1j*np.conjugate(Iseq))/np.imag(Z1l*Ig*1j*np.conjugate(Iseq))
            print("Distancia(pu): "+str(m))
        else:
            print("Falla CG")
            Vg=Vc
            Ig=Ic+k0*I0*3
            Iseq=I2*cmath.rect(1,-2*np.pi/3)
            m=np.imag(Vg*np.conjugate(Iseq))/np.imag(Z1l*Ig*np.conjugate(Iseq))
            print("Distancia(pu): "+str(m))
    elif msg=="CG o alguna bifasica a tierra con Rf alto":
        #Calculo Rf
        Rcg=np.abs(np.imag(Vc*np.conjugate(Z1l*(Ic+k0*3*I0)))/np.imag(1.5*(I2+I0)*np.conjugate(Z1l*(Ic+k0*3*I0))))
        Rab=np.abs(np.imag((Va-Vb)*np.conjugate(Z1l*((Ia-Ib)+k0*3*I0)))/np.imag(1.5*(I2+I0)*np.conjugate(Z1l*((Ia-Ib)+k0*3*I0))))
        Rac=np.abs(np.imag((Va-Vc)*np.conjugate(Z1l*((Ia-Ic)+k0*3*I0)))/np.imag(1.5*(I2+I0)*np.conjugate(Z1l*((Ia-Ic)+k0*3*I0))))
        Rcb=np.abs(np.imag((Vc-Vb)*np.conjugate(Z1l*((Ic-Ib)+k0*3*I0)))/np.imag(1.5*(I2+I0)*np.conjugate(Z1l*((Ic-Ib)+k0*3*I0))))
        #calculo MHO
        Mab=np.abs((Va-Vb)/(Ia-Ib))
        Mac=np.abs((Va-Vc)/(Ia-Ic))
        Mcb=np.abs((Vc-Vb)/(Ic-Ib))

        if min(Mab,Mac,Mcb)==Mab:
            if Rcg<Rab:
                print("Falla c-g")
                Vg=Vc
                Ig=Ic+k0*I0*3
                Iseq=I2*cmath.rect(1,-2*np.pi/3)
                m=np.imag(Vg*np.conjugate(Iseq))/np.imag(Z1l*Ig*np.conjugate(Iseq))
                print("Distancia(pu): "+str(m))
            else:
                print("Falla a-b")
                Vg=Va-Vb
                Ig=Ia-Ib
                Iseq=I2*cmath.rect(1,-2*np.pi/3)
                m=np.imag(Vg*1j*np.conjugate(Iseq))/np.imag(Z1l*Ig*1j*np.conjugate(Iseq))
                print("Distancia(pu): "+str(m))
        elif min(Mab,Mac,Mcb)==Mac:
            if Rcg<Rac:
                print("Falla c-g")
                Vg=Vc
                Ig=Ic+k0*I0*3
                Iseq=I2*cmath.rect(1,-2*np.pi/3)
                m=np.imag(Vg*np.conjugate(Iseq))/np.imag(Z1l*Ig*np.conjugate(Iseq))
                print("Distancia(pu): "+str(m))
            else:
                print("Falla a-c")       
                Vg=Vc-Va
                Ig=Ic-Ia
                Iseq=I2*cmath.rect(1,2*np.pi/3)#Referenciada a la fase sana (B)
                m=np.imag(Vg*1j*np.conjugate(Iseq))/np.imag(Z1l*Ig*1j*np.conjugate(Iseq))
                print("Distancia(pu): "+str(m))
        else:
            if Rcg<Rcb:
                print("Falla c-g")
                Vg=Vc
                Ig=Ic+k0*I0*3
                Iseq=I2*cmath.rect(1,-2*np.pi/3)
                m=np.imag(Vg*np.conjugate(Iseq))/np.imag(Z1l*Ig*np.conjugate(Iseq))
                print("Distancia(pu): "+str(m))
            else:
                print("Falla c-b")
                Vg=Vb-Vc
                Ig=Ib-Ic
                Iseq=I2
                m=np.imag(Vg*1j*np.conjugate(Iseq))/np.imag(Z1l*Ig*1j*np.conjugate(Iseq))
                print("Distancia(pu): "+str(m))
    elif msg=="BG o alguna bifasica a tierra con Rf alto":
        #Calculo Rf
        Rbg=np.abs(np.imag(Vb*np.conjugate(Z1l*(Ib+k0*3*I0)))/np.imag(1.5*(I2+I0)*np.conjugate(Z1l*(Ib+k0*3*I0))))
        Rab=np.abs(np.imag((Va-Vb)*np.conjugate(Z1l*((Ia-Ib)+k0*3*I0)))/np.imag(1.5*(I2+I0)*np.conjugate(Z1l*((Ia-Ib)+k0*3*I0))))
        Rac=np.abs(np.imag((Va-Vc)*np.conjugate(Z1l*((Ia-Ic)+k0*3*I0)))/np.imag(1.5*(I2+I0)*np.conjugate(Z1l*((Ia-Ic)+k0*3*I0))))
        Rcb=np.abs(np.imag((Vc-Vb)*np.conjugate(Z1l*((Ic-Ib)+k0*3*I0)))/np.imag(1.5*(I2+I0)*np.conjugate(Z1l*((Ic-Ib)+k0*3*I0))))
        #calculo MHO
        Mab=np.abs((Va-Vb)/(Ia-Ib))
        Mac=np.abs((Va-Vc)/(Ia-Ic))
        Mcb=np.abs((Vc-Vb)/(Ic-Ib))

        if min(Mab,Mac,Mcb)==Mab:
            if Rbg<Rab:
                print("Falla b-g")
                Vg=Vb
                Ig=Ib+k0*I0*3
                Iseq=I2*cmath.rect(1,2*np.pi/3)
                m=np.imag(Vg*np.conjugate(Iseq))/np.imag(Z1l*Ig*np.conjugate(Iseq))
                print("Distancia(pu): "+str(m))
            else:
                print("Falla a-b")
                Vg=Va-Vb
                Ig=Ia-Ib
                Iseq=I2*cmath.rect(1,-2*np.pi/3)
                m=np.imag(Vg*1j*np.conjugate(Iseq))/np.imag(Z1l*Ig*1j*np.conjugate(Iseq))
                print("Distancia(pu): "+str(m))
        elif min(Mab,Mac,Mcb)==Mac:
            if Rbg<Rac:
                print("Falla b-g")
                Vg=Vb
                Ig=Ib+k0*I0*3
                Iseq=I2*cmath.rect(1,2*np.pi/3)
                m=np.imag(Vg*np.conjugate(Iseq))/np.imag(Z1l*Ig*np.conjugate(Iseq))
                print("Distancia(pu): "+str(m))
            else:
                print("Falla a-c")
                Vg=Vc-Va
                Ig=Ic-Ia
                Iseq=I2*cmath.rect(1,2*np.pi/3)#Referenciada a la fase sana (B)
                m=np.imag(Vg*1j*np.conjugate(Iseq))/np.imag(Z1l*Ig*1j*np.conjugate(Iseq))
                print("Distancia(pu): "+str(m))
        else:
            if Rbg<Rcb:
                print("Falla b-g")
                Vg=Vb
                Ig=Ib+k0*I0*3
                Iseq=I2*cmath.rect(1,2*np.pi/3)
                m=np.imag(Vg*np.conjugate(Iseq))/np.imag(Z1l*Ig*np.conjugate(Iseq))
                print("Distancia(pu): "+str(m))
            else:
                print("Falla c-b")
                Vg=Vb-Vc
                Ig=Ib-Ic
                Iseq=I2
                m=np.imag(Vg*1j*np.conjugate(Iseq))/np.imag(Z1l*Ig*1j*np.conjugate(Iseq))
                print("Distancia(pu): "+str(m))
    elif msg=="AG o alguna bifasica a tierra con Rf alto":
        #Calculo Rf
        Rag=np.abs(np.imag(Va*np.conjugate(Z1l*(Ia+k0*3*I0)))/np.imag(1.5*(I2+I0)*np.conjugate(Z1l*(Ia+k0*3*I0))))
        Rab=np.abs(np.imag((Va-Vb)*np.conjugate(Z1l*((Ia-Ib)+k0*3*I0)))/np.imag(1.5*(I2+I0)*np.conjugate(Z1l*((Ia-Ib)+k0*3*I0))))
        Rac=np.abs(np.imag((Va-Vc)*np.conjugate(Z1l*((Ia-Ic)+k0*3*I0)))/np.imag(1.5*(I2+I0)*np.conjugate(Z1l*((Ia-Ic)+k0*3*I0))))
        Rcb=np.abs(np.imag((Vc-Vb)*np.conjugate(Z1l*((Ic-Ib)+k0*3*I0)))/np.imag(1.5*(I2+I0)*np.conjugate(Z1l*((Ic-Ib)+k0*3*I0))))
        #calculo MHO
        Mab=np.abs((Va-Vb)/(Ia-Ib))
        Mac=np.abs((Va-Vc)/(Ia-Ic))
        Mcb=np.abs((Vc-Vb)/(Ic-Ib))

        if min(Mab,Mac,Mcb)==Mab:
            if Rag<Rab:
                print("Falla a-g")
                Vg=Va
                Ig=Ia+k0*I0*3
                Iseq=I2
                m=np.imag(Vg*np.conjugate(Iseq))/np.imag(Z1l*Ig*np.conjugate(Iseq))
                print("Distancia(pu): "+str(m))
            else:
                print("Falla a-b")
                Vg=Va-Vb
                Ig=Ia-Ib
                Iseq=I2*cmath.rect(1,-2*np.pi/3)
                m=np.imag(Vg*1j*np.conjugate(Iseq))/np.imag(Z1l*Ig*1j*np.conjugate(Iseq))
                print("Distancia(pu): "+str(m))
        elif min(Mab,Mac,Mcb)==Mac:
            if Rag<Rac:
                print("Falla a-g")
                Vg=Va
                Ig=Ia+k0*I0*3
                Iseq=I2
                m=np.imag(Vg*np.conjugate(Iseq))/np.imag(Z1l*Ig*np.conjugate(Iseq))
                print("Distancia(pu): "+str(m))
            else:
                print("Falla a-c")
                Vg=Vc-Va
                Ig=Ic-Ia
                Iseq=I2*cmath.rect(1,2*np.pi/3)#Referenciada a la fase sana (B)
                m=np.imag(Vg*1j*np.conjugate(Iseq))/np.imag(Z1l*Ig*1j*np.conjugate(Iseq))
                print("Distancia(pu): "+str(m))
        else:
            if Rag<Rcb:
                print("Falla a-g")
                Vg=Va
                Ig=Ia+k0*I0*3
                Iseq=I2
                m=np.imag(Vg*np.conjugate(Iseq))/np.imag(Z1l*Ig*np.conjugate(Iseq))
                print("Distancia(pu): "+str(m))
            else:
                print("Falla c-b")
                Vg=Vb-Vc
                Ig=Ib-Ic
                Iseq=I2
                m=np.imag(Vg*1j*np.conjugate(Iseq))/np.imag(Z1l*Ig*1j*np.conjugate(Iseq))
                print("Distancia(pu): "+str(m))
    elif msg=="Falla trifasica":
        print("ABC")
        Vg=Va-Vb
        Ig=Ia-Ib
        m=np.imag(Vg/Ig)/np.imag(Z1l)
        print("Distancia(pu): "+str(m))
    else:
        print("Sin falla")   

#Determina si hay falla y da una aproximacion de cuales pueden ser las fases falladas.
def Evaluar(I0,I1,I2,I_nominal,I_sc3_min):
    #print(np.abs(I0),np.abs(I2),np.abs(I1))
    #print(I0,I1,I2)
    if(np.abs(I0)/np.abs(I1)<0.005 and np.abs(I2)/np.abs(I1)<0.005 and np.abs(I1)>=I_sc3_min):
            return "Falla trifasica"
    elif((np.angle(I2,True))%360<=(np.angle(I0,True)-90)%360 and (np.angle(I2,True))%360>=(np.angle(I0,True)-150)%360):
            return "Falla BG o AC"
    elif((np.angle(I2,True))%360<=(np.angle(I0,True)+30)%360 and (np.angle(I2,True))%360>=(np.angle(I0,True)-30)%360):
            return "Falla AG o BC"
    elif((np.angle(I2,True))%360<=(np.angle(I0,True)+150)%360 and (np.angle(I2,True))%360>=(np.angle(I0,True)+90)%360):
            return "Falla CG o BA"
    elif(((np.angle(I2,True))%360<=(np.angle(I0,True)+60)%360 and (np.angle(I2,True))%360>=(np.angle(I0,True)+30)%360) or (((np.angle(I2,True))%360<=(np.angle(I0,True)-30)%360 and (np.angle(I2,True))%360>=(np.angle(I0,True)-60)%360))):
            return "AG o alguna bifasica a tierra con Rf alto"
    elif(((np.angle(I2,True))%360<=(np.angle(I0,True)-60)%360 and (np.angle(I2,True))%360>=(np.angle(I0,True)-90)%360) or (((np.angle(I2,True))%360<=(np.angle(I0,True)-150)%360 and (np.angle(I2,True))%360>=(np.angle(I0,True)-180)%360))):
            return "BG o alguna bifasica a tierra con Rf alto"
    elif(((np.angle(I2,True))%360<=(np.angle(I0,True)+90)%360 and (np.angle(I2,True))%360>=(np.angle(I0,True)+60)%360) or (((np.angle(I2,True))%360<=(np.angle(I0,True)+180)%360 and (np.angle(I2,True))%360>=(np.angle(I0,True)+150)%360))):
            return "CG o alguna bifasica a tierra con Rf alto"
    else: 
            return "Sin falla"

#Retorna el fasor de 60Hz de la señal dada como input
def Espectro(data,fs):
    #Filtro pasa bajas
    cutoff = (fs-60)/3      # desired cutoff frequency of the filter, Hz
    order = 2

    y=Filtro(fs,order,cutoff,data)
    
    #Transformada
    N=len(data)
    Tf=1/fs
    SOL=FFT(y)
    SOL_f=FFT_f(N,Tf)

    SOL_abs=2.0/N * np.abs(SOL[0:N//2])
    SOL_angle=np.angle(SOL)[0:N//2]
    i=closest(SOL_f,60)
    j=np.where(SOL_f==i)#Index con el fasor de 60Hz
    #plt.polar([0,SOL_angle[j]],[0,SOL_abs[j]],marker='.')

    return cmath.rect(SOL_abs[j],SOL_angle[j])

#Valores dados
I_nominal=800
I_sc3_min=27370#pico
Z0l=(0.283+0.8226j)
Z1l=(0.06988+0.2608j)
K0=abs((1/3)*(Z0l/Z1l-1))

Windowing(senal_I_a,senal_I_b,senal_I_c,rec,I_nominal,I_sc3_min,senal_V_a,senal_V_b,senal_V_c,K0,Z1l)


Trigger time = 0.0s
Falla c-b
Distancia(pu): 0.5367254452351019
Falla c-b
Distancia(pu): 0.5272801392688008
Falla c-b
Distancia(pu): 0.5309102524366653
Falla c-b
Distancia(pu): 0.546976769884049
Falla c-b
Distancia(pu): 0.5728226607107864
Falla c-b
Distancia(pu): 0.6023838008785712
Falla c-b
Distancia(pu): 0.626533922607235
Falla c-b
Distancia(pu): 0.637245043538038
Falla c-b
Distancia(pu): 0.6319629424088711
Falla c-b
Distancia(pu): 0.6147291570846133
Falla c-b
Distancia(pu): 0.5925069355054411
Falla c-b
Distancia(pu): 0.5719322904060324
Falla c-b
Distancia(pu): 0.5576497378923911
Falla c-b
Distancia(pu): 0.551987379823087
Falla c-b
Distancia(pu): 0.555276869263742
Falla c-b
Distancia(pu): 0.5658997283261878
Falla c-b
Distancia(pu): 0.5807159192323311
Falla c-b
Distancia(pu): 0.5957599008666308
Falla c-b
Distancia(pu): 0.6074170169309829
Falla c-b
Distancia(pu): 0.6130451374106347
Falla c-b
Distancia(pu): 0.6115235818401763
Falla c-b
Distancia(pu): 0.6033904099301728
Falla c-b
Distanc